# Video Subtitle Generation using Whisper and OpenVINO™

[Whisper](https://openai.com/blog/whisper/) is an automatic speech recognition (ASR) system trained on 680,000 hours of multilingual and multitask supervised data collected from the web. It is a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

![asr-training-data-desktop.svg](https://user-images.githubusercontent.com/29454499/204536347-28976978-9a07-416c-acff-fc1214bbfbe0.svg)

You can find more information about this model in the [research paper](https://cdn.openai.com/papers/whisper.pdf), [OpenAI blog](https://openai.com/blog/whisper/), [model card](https://github.com/openai/whisper/blob/main/model-card.md) and GitHub [repository](https://github.com/openai/whisper).

In this notebook, we will use Whisper with OpenVINO to generate subtitles in a sample video.
Notebook contains the following steps:
1. Download the model.
2. Instantiate the PyTorch model pipeline.
3. Export the ONNX model and convert it to OpenVINO IR, using the Model Optimizer tool.
4. Run the Whisper pipeline with OpenVINO models.

## Prerequisites

Clone and install the model repository.

There are several models of different sizes and capabilities trained by the authors of the model. In this tutorial, we will use the `base` model, but the same actions are also applicable to other models from Whisper family.

In [1]:
import whisper

model = whisper.load_model("small")
model.to("cpu")
model.eval()
pass

### Convert model to OpenVINO Intermediate Representation (IR) format.

For best results with OpenVINO, it is recommended to convert the model to OpenVINO IR format. OpenVINO supports PyTorch via ONNX conversion. We will use `torch.onnx.export` for exporting the ONNX model from PyTorch. We need to provide initialized model object and example of inputs for shape inference. We will use `mo.convert_model` functionality to convert the ONNX models. The `mo.convert_model` Python function returns an OpenVINO model ready to load on device and start making predictions. We can save it on disk for next usage with `openvino.runtime.serialize`.


### Convert Whisper Encoder to OpenVINO IR

In [2]:
from pathlib import Path

WHISPER_ENCODER_OV = Path(f"whisper_encoder.xml")
WHISPER_DECODER_OV = Path(f"whisper_decoder.xml")

### Convert Whisper decoder to OpenVINO IR

To reduce computational complexity, the decoder uses cached key/value projections in attention modules from the previous steps. We need to modify this process for correct tracing to ONNX.

The decoder model autoregressively predicts the next token guided by encoder hidden states and previously predicted sequence. This means that the shape of inputs which depends on the previous step (inputs for tokens and attention hidden states from previous step) are dynamic. For efficient utilization of memory, you define an upper bound for dynamic input shapes.

## Prepare inference pipeline

The image below illustrates the pipeline of video transcribing using the Whisper model.

![whisper_pipeline.png](https://user-images.githubusercontent.com/29454499/204536733-1f4342f7-2328-476a-a431-cb596df69854.png)

To run the PyTorch Whisper model, we just need to call the `model.transcribe(audio, **parameters)` function. We will try to reuse original model pipeline for audio transcribing after replacing the original models with OpenVINO IR versions.

In [3]:
import torch
from typing import Optional, Union, List, Dict
from functools import partial


class OpenVINOAudioEncoder(torch.nn.Module):
    """
    Helper for inference Whisper encoder model with OpenVINO
    """

    def __init__(self, core, model_path, device='CPU'):
        super().__init__()
        self.model = core.read_model(model_path)
        self.compiled_model = core.compile_model(self.model, device)
        self.output_blob = self.compiled_model.output(0)

    def forward(self, mel: torch.Tensor):
        """
        Inference OpenVINO whisper encoder model.

        Parameters:
          mel: input audio fragment mel spectrogram.
        Returns:
          audio_features: torch tensor with encoded audio features.
        """
        return torch.from_numpy(self.compiled_model(mel)[self.output_blob])

In [4]:
from openvino.runtime import Core, Tensor


class OpenVINOTextDecoder(torch.nn.Module):
    """
    Helper for inference OpenVINO decoder model
    """

    def __init__(self, core: Core, model_path: Path, device: str = 'CPU'):
        super().__init__()
        self._core = core
        self.model = core.read_model(model_path)
        self._input_names = [inp.any_name for inp in self.model.inputs]
        self.compiled_model = core.compile_model(self.model, device)
        self.device = device

    def init_past_inputs(self, feed_dict):
        """
        Initialize cache input for first step.

        Parameters:
          feed_dict: Dictonary with inputs for inference
        Returns:
          feed_dict: updated feed_dict
        """
        beam_size = feed_dict['tokens'].shape[0]
        audio_len = feed_dict['audio_features'].shape[2]
        previous_seq_len = 0
        for name in self._input_names:
            if name in ['tokens', 'audio_features']:
                continue
            feed_dict[name] = Tensor(np.zeros(
                (beam_size, previous_seq_len, audio_len), dtype=np.float32))
        return feed_dict

    def preprocess_kv_cache_inputs(self, feed_dict, kv_cache):
        """
        Transform kv_cache to inputs

        Parameters:
          feed_dict: dictionary with inputs for inference
          kv_cache: dictionary with cached attention hidden states from previous step
        Returns:
          feed_dict: updated feed dictionary with additional inputs
        """
        if not kv_cache:
            return self.init_past_inputs(feed_dict)
        for k, v in kv_cache.items():
            new_k = f'in_{k}'
            if new_k in self._input_names:
                feed_dict[new_k] = Tensor(v.numpy())
        return feed_dict

    def postprocess_outputs(self, outputs):
        """
        Transform model output to format expected by the pipeline

        Parameters:
          outputs: outputs: raw inference results.
        Returns:
          logits: decoder predicted token logits
          kv_cache: cached attention hidden states
        """
        logits = None
        kv_cache = {}
        for output_t, out in outputs.items():
            if 'logits' in output_t.get_names():
                logits = torch.from_numpy(out)
            else:
                tensor_name = output_t.any_name
                kv_cache[tensor_name.replace(
                    'out_', '')] = torch.from_numpy(out)
        return logits, kv_cache

    def forward(self, x: torch.Tensor, xa: torch.Tensor, kv_cache: Optional[dict] = None):
        """
        Inference decoder model.

        Parameters:
          x: torch.LongTensor, shape = (batch_size, <= n_ctx) the text tokens
          xa: torch.Tensor, shape = (batch_size, n_mels, n_audio_ctx)
             the encoded audio features to be attended on
          kv_cache: Dict[str, torch.Tensor], attention modules hidden states cache from previous steps
        Returns:
          logits: decoder predicted logits
          kv_cache: updated kv_cache with current step hidden states
        """
        feed_dict = {'tokens': Tensor(x.numpy()), 'audio_features': Tensor(xa.numpy())}
        feed_dict = (self.preprocess_kv_cache_inputs(feed_dict, kv_cache))
        res = self.compiled_model(feed_dict)
        return self.postprocess_outputs(res)

In [5]:
from whisper.decoding import DecodingTask, Inference, DecodingOptions, DecodingResult


class OpenVINOInference(Inference):
    """
    Wrapper for inference interface
    """

    def __init__(self, model: "Whisper", initial_token_length: int):
        self.model: "Whisper" = model
        self.initial_token_length = initial_token_length
        self.kv_cache = {}

    def logits(self, tokens: torch.Tensor, audio_features: torch.Tensor) -> torch.Tensor:
        """
        getting logits for given tokens sequence and audio features and save kv_cache

        Parameters:
          tokens: input tokens
          audio_features: input audio features
        Returns:
          logits: predicted by decoder logits
        """
        if tokens.shape[-1] > self.initial_token_length:
            # only need to use the last token except in the first forward pass
            tokens = tokens[:, -1:]
        logits, self.kv_cache = self.model.decoder(
            tokens, audio_features, kv_cache=self.kv_cache)
        return logits

    def cleanup_caching(self):
        """
        Reset kv_cache to initial state
        """
        self.kv_cache = {}

    def rearrange_kv_cache(self, source_indices):
        """
        Update hidden states cache for selected sequences
        Parameters:
          source_indicies: sequences indicies
        Returns:
          None
        """
        for module, tensor in self.kv_cache.items():
            # update the key/value cache to contain the selected sequences
            self.kv_cache[module] = tensor[source_indices]


class OpenVINODecodingTask(DecodingTask):
    """
    Class for decoding using OpenVINO
    """

    def __init__(self, model: "Whisper", options: DecodingOptions):
        super().__init__(model, options)
        self.inference = OpenVINOInference(model, len(self.initial_tokens))


@torch.no_grad()
def decode(model: "Whisper", mel: torch.Tensor, options: DecodingOptions = DecodingOptions()) -> Union[DecodingResult, List[DecodingResult]]:
    """
    Performs decoding of 30-second audio segment(s), provided as Mel spectrogram(s).

    Parameters
    ----------
    model: Whisper
        the Whisper model instance

    mel: torch.Tensor, shape = (80, 3000) or (*, 80, 3000)
        A tensor containing the Mel spectrogram(s)

    options: DecodingOptions
        A dataclass that contains all necessary options for decoding 30-second segments

    Returns
    -------
    result: Union[DecodingResult, List[DecodingResult]]
        The result(s) of decoding contained in `DecodingResult` dataclass instance(s)
    """
    single = mel.ndim == 2
    if single:
        mel = mel.unsqueeze(0)

    result = OpenVINODecodingTask(model, options).run(mel)

    if single:
        result = result[0]

    return result

In [6]:
del model.decoder
del model.encoder

In [8]:
from collections import namedtuple

Parameter = namedtuple('Parameter', ['device'])

core = Core()

model.encoder = OpenVINOAudioEncoder(core, 'whisper/whisper_encoder.xml')
model.decoder = OpenVINOTextDecoder(core, 'whisper/whisper_decoder.xml')
model.decode = partial(decode, model)


def parameters():
    return iter([Parameter(torch.device('cpu'))])


model.parameters = parameters


def logits(model, tokens: torch.Tensor, audio_features: torch.Tensor):
    """
    Override for logits extraction method
    Parameters:
      toekns: input tokens
      audio_features: input audio features
    Returns:
      logits: decoder predicted logits
    """
    return model.decoder(tokens, audio_features, None)[0]


model.logits = partial(logits, model)

#### Define audio preprocessing

The model expects mono-channel audio with a 16000 Hz sample rate, represented in floating point range. When the audio from the input video does not meet these requirements, we will need to apply preprocessing.

In [9]:
import io
from pathlib import Path
import numpy as np
from scipy.io import wavfile
from pytube import YouTube
from moviepy.editor import VideoFileClip


def resample(audio, src_sample_rate, dst_sample_rate):
    """
    Resample audio to specific sample rate

    Parameters:
      audio: input audio signal
      src_sample_rate: source audio sample rate
      dst_sample_rate: destination audio sample rate
    Returns:
      resampled_audio: input audio signal resampled with dst_sample_rate
    """
    if src_sample_rate == dst_sample_rate:
        return audio
    duration = audio.shape[0] / src_sample_rate
    resampled_data = np.zeros(shape=(int(duration * dst_sample_rate)), dtype=np.float32)
    x_old = np.linspace(0, duration, audio.shape[0], dtype=np.float32)
    x_new = np.linspace(0, duration, resampled_data.shape[0], dtype=np.float32)
    resampled_audio = np.interp(x_new, x_old, audio)
    return resampled_audio.astype(np.float32)


def audio_to_float(audio):
    """
    convert audio signal to floating point format
    """
    return audio.astype(np.float32) / np.iinfo(audio.dtype).max


def get_audio(video_file):
    """
    Extract audio signal from a given video file, then convert it to float, 
    then mono-channel format and resample it to the expected sample rate

    Parameters:
        video_file: path to input video file
    Returns:
      resampled_audio: mono-channel float audio signal with 16000 Hz sample rate 
                       extracted from video  
    """
    input_video = VideoFileClip(str(video_file))
    input_video.audio.write_audiofile(video_file.stem + '.wav', verbose=False, logger=None)
    input_audio_file = video_file.stem + '.wav'
    sample_rate, audio = wavfile.read(
        io.BytesIO(open(input_audio_file, 'rb').read()))
    audio = audio_to_float(audio)
    if audio.ndim == 2:
        audio = audio.mean(axis=1)
    resampled_audio = resample(audio, sample_rate, 16000)
    return resampled_audio

## Run video transcription pipeline

Now, we are ready to start transcription. We select a video from YouTube that we want to transcribe. Be patient, as downloading the video may take some time.

In [10]:
import ipywidgets as widgets
VIDEO_LINK = "https://www.youtube.com/watch?v=A7OneT71EGs"
link = widgets.Text(
    value=VIDEO_LINK,
    placeholder="Type link for video",
    description="Video:",
    disabled=False
)

link

Text(value='https://www.youtube.com/watch?v=A7OneT71EGs', description='Video:', placeholder='Type link for vid…

In [11]:
print(f"Downloading video {link.value} started")

output_file = Path("downloaded_video.mp4")
yt = YouTube(link.value)
yt.streams.get_highest_resolution().download(filename=output_file)
print(f"Video saved to {output_file}")

Video saved to downloaded_video.mp4


Select the task for the model:
* **transcribe** - generate audio transcription in the source language (automatically detected).
* **translate** - generate audio transcription with translation to English language.

In [12]:
task = widgets.Select(
    options=["transcribe", "translate"],
    value="transcribe",
    description="Select task:",
    disabled=False
)
task

Select(description='Select task:', options=('transcribe', 'translate'), value='transcribe')

In [13]:
from moviepy.editor import VideoFileClip
import os
import shutil
    
def split_video(input_file, output_dir, duration=300):
    # 確保輸出目錄存在並清空目錄內容
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # 刪除目錄及其內容
    os.makedirs(output_dir)  # 創建空目錄

    # 讀取原始影片
    clip = VideoFileClip(input_file)

    # 計算影片長度（單位：秒）
    total_duration = clip.duration

    audio_files = []  # 存儲切割出的語音檔案路徑

    # 切割影片
    for i in range(0, int(total_duration), duration):
        # 計算切割起始和結束時間
        start_time = i
        end_time = min(i + duration, total_duration)

        # 提取子片段
        subclip = clip.subclip(start_time, end_time)

        # 生成子片段的文件名
        output_file = os.path.join(output_dir, f"segment_{start_time}-{end_time}.mp4")

        # 將子片段保存為文件
        subclip.write_videofile(output_file, codec="libx264")

        # 將語音檔案路徑添加到列表中
        audio_files.append(output_file)

    # 釋放資源
    clip.close()

    return audio_files

# 假設 get_audio 函數已經定義好了，用於獲取語音檔案路徑

# 使用範例
input_file = "downloaded_video.mp4"  # 輸入影片文件
output_dir = "output"  # 輸出目錄
audio_files = split_video(input_file, output_dir)
print(audio_files)

Moviepy - Building video output/segment_0-283.28.mp4.
MoviePy - Writing audio in segment_0-283.28TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output/segment_0-283.28.mp4



Moviepy - Done !
Moviepy - video ready output/segment_0-283.28.mp4
['output/segment_0-283.28.mp4']


In [14]:
output_file_path = output_file.with_suffix(".txt")

with output_file_path.open("w") as f:
        pass  # 空操作，不寫入任何內容

# 在上面的 for 迴圈中遍歷語音檔案並進行處理
for audio_file in audio_files:
    divide_mp4 = Path(audio_file)
    audio = get_audio(divide_mp4)
    # 在這裡添加您對語音檔案的處理邏輯
    transcription = model.transcribe(audio, language="Chinese",initial_prompt="以下是普通話的句子。", beam_size=5, best_of=5, task=task.value)
    
    # 將轉錄的文字附加到文本文件中
    with output_file_path.open("a") as f:  # 以附加模式打開文件
        f.write(transcription["text"] + "\n")  # 寫入轉錄的文字

In [15]:
def format_timestamp(seconds: float):
    """
    format time in srt-file excpected format
    """
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    return (f"{hours}:" if hours > 0 else "00:") + f"{minutes:02d}:{seconds:02d},{milliseconds:03d}"


def prepare_srt(transcription):
    """
    Format transcription into srt file format
    """
    segment_lines = []
    for segment in transcription["segments"]:
        segment_lines.append(str(segment["id"] + 1) + "\n")
        time_start = format_timestamp(segment["start"])
        time_end = format_timestamp(segment["end"])
        time_str = f"{time_start} --> {time_end}\n"
        segment_lines.append(time_str)
        segment_lines.append(segment["text"] + "\n\n")
    return segment_lines

"The results will be saved in the `downloaded_video.srt` file. SRT is one of the most popular formats for storing subtitles and is compatible with many modern video players. This file can be used to embed transcription into videos during playback or by injecting them directly into video files using `ffmpeg`.

In [16]:
srt_lines = prepare_srt(transcription)
# save transcription
with output_file.with_suffix(".srt").open("w") as f:
    f.writelines(srt_lines)

Now let us see the results.

## JiWER

JiWER is a simple and fast python package to evaluate an automatic speech recognition system. It supports the following measures:

1. word error rate (WER)
2. match error rate (MER)
3. word information lost (WIL)
4. word information preserved (WIP)
5. character error rate (CER)
   
These measures are computed with the use of the minimum-edit distance between one or more reference and hypothesis sentences. The minimum-edit distance is calculated using RapidFuzz, which uses C++ under the hood, and is therefore faster than a pure python implementation.

[back to top ⬆️](#Table-of-contents:)


In [17]:
import jiwer
import jieba

class ChineseTextProcessor:
    @staticmethod
    def tokenize(text):
        # 使用jieba分詞
        return jieba.lcut(text)

    @staticmethod
    def join_words_with_spaces(word_sequence):
        # 使用空格將單詞序列連接起來
        return ' '.join(word_sequence)

class WERCalculator:
    def __init__(self, hypothesis, ground_truth):
        self.hypothesis = hypothesis
        self.ground_truth = ground_truth
    
    def preprocess_text(self, text):
        text = jiwer.RemoveWhiteSpace()(text)
        text = jiwer.RemoveMultipleSpaces()(text)
        text = jiwer.RemovePunctuation()(text)
        return text
    
    def calculate_wer(self):
        # 做文字預處理
        hypothesis = self.preprocess_text(self.hypothesis)
        ground_truth = self.preprocess_text(self.ground_truth)
        
        # 將中文文本轉換為單詞序列
        hypothesis_words = ChineseTextProcessor.tokenize(hypothesis)
        hypothesis_text = ChineseTextProcessor.join_words_with_spaces(hypothesis_words)

        ground_truth_words = ChineseTextProcessor.tokenize(ground_truth)
        ground_truth_text = ChineseTextProcessor.join_words_with_spaces(ground_truth_words)

        # 計算 WER
        output_wer = jiwer.compute_measures(ground_truth_text, hypothesis_text)
        output_cer = jiwer.process_characters(ground_truth_text, hypothesis_text)
        print("ground_truth:/n",ground_truth_text)
        print("--------------------------------------------")
        print("hypothesis:/n",hypothesis_text)
        return output_wer,output_cer

if __name__ == '__main__':
    # 讀取測試文本和答案文本
    with open('downloaded_video.txt', 'r') as f:
        test = f.read()

    with open('../answer.txt', 'r') as a:
        answer = a.read()

    # 初始化 WERCalculator 物件並計算 WER
    calculator = WERCalculator(test, answer)
    result_wer,result_cer = calculator.calculate_wer()

    print("WER:", result_wer['wer'])
    print("MER:", result_wer['mer'])
    print("WIL:", result_wer['wil'])
    print("WIP:", result_wer['wip'])
    print("CER:", result_cer.cer)


FileNotFoundError: [Errno 2] No such file or directory: '../answer.txt'